In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
pip install pickle5

In [4]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy 
import pickle
from glob import glob
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [40]:
from glob import glob
import pickle5 as pickle
import numpy as np

ROOT_PATH = "../input/"

cities = ["austin", "miami", "pittsburgh", "dearborn", "washington-dc", "palo-alto"]
splits = ["train", "test"]

def get_city_trajectories(city="palo-alto", split="train", normalized=False):
    if split=="train":
        f_in = ROOT_PATH + split + "/" + city + "_inputs"
        inputs = pickle.load(open(f_in, "rb"))
        inputs = np.asarray(inputs).astype(np.float64)

        outputs = None
    
        f_out = ROOT_PATH + split + "/" + city + "_outputs"
        outputs = pickle.load(open(f_out, "rb"))
        outputs = np.asarray(outputs).astype(np.float64)
        
    elif split=="test":
        f_in = ROOT_PATH + "test-inputs" + "/" + city + "_inputs"
        inputs = pickle.load(open(f_in, "rb"))
        inputs = np.asarray(inputs).astype(np.float64)

        outputs = None
        
    return inputs, outputs

class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, city: str, split:str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.transform = transform

        self.inputs, self.outputs = get_city_trajectories(city=city, split=split, normalized=False)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):

        data = (self.inputs[idx], self.outputs[idx])
            
        if self.transform:
            data = self.transform(data)

        return data

# intialize a dataset
city = 'palo-alto' 
split = 'train'
train_dataset  = ArgoverseDataset(city = city, split = split)

In [33]:
# convert the original training document into trainable batches 
def get_batches(inp, out, batch_size):
    
    num_batches = int(len(inp) / (batch_size))
    
    X = inp[:num_batches*batch_size]
    Y = out[:num_batches*batch_size]
#     Y = np.zeros((len(X), 50))
    
    
    # generate trainable batches
    for i in range(0, num_batches*batch_size, batch_size):
        Xs = []
        Ys = []
        for j in range(i, i+batch_size):
            temp_x = X[j]
               
            temp_y = Y[j]
            
            Xs.append(temp_x)
            Ys.append(temp_y)
        
        Xs = np.array(Xs)
        Ys = np.array(Ys)

        yield torch.tensor(Xs), torch.tensor(Ys)

In [34]:
import torch.nn as nn

In [64]:
class Pred(nn.Module):

    def __init__(self):
        super().__init__()
        
        self.encoder = nn.Sequential(
            nn.Linear(100, 64),
            nn.ReLU(),
#             nn.Linear(64, 32),
            nn.Linear(64, 16)
#             nn.ReLU(),
#             nn.Linear(32, 16),
#             nn.ReLU(),
#             nn.Linear(16, 16)
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(16, 120),
            nn.ReLU(),
#             nn.Linear(32, 64),
#             nn.ReLU(),
#             nn.Linear(64, 128),
#             nn.ReLU(),
#             nn.Linear(128, 120),
#             nn.ReLU(),
            nn.Linear(120, 120)
        )
        
    def forward(self, x):
        x = x.reshape(-1, 100).float()
        x = self.encoder(x)
        x = self.decoder(x)
#         print("------")
#         print(x.shape)
        x = x.reshape(-1, 60, 2)
        return x

In [76]:
pred = Pred()
opt = torch.optim.Adam(pred.parameters(), lr=0.0001)

In [75]:
batch_sz = 512  # batch size 
city = 'palo-alto' 
split = 'train'
train_dataset  = ArgoverseDataset(city = city, split = split)
train_loader = DataLoader(train_dataset,batch_size=batch_sz)

In [77]:
def train(pred):
    for epoch in range(5000):
        total_loss = 0
        i = 0
        for i_batch, sample_batch in enumerate(train_loader):
            inp, out = sample_batch
#             print(inp.shape)
            preds = pred(inp)
#             print(preds.shape)
#             print(out.shape)
            loss = ((preds - out) ** 2).sum()

            opt.zero_grad()
            loss.backward()
            opt.step()
            
            i += len(inp) * 60 * 2
            
            total_loss += loss.item()
            
        print('epoch {} loss: {}'.format(epoch, total_loss / i))
    return pred

In [78]:
import time

In [79]:
start = time.time()
print("Starts training process")
model = train(pred)
end = time.time()
print("Ends training process, total time - " + str(end - start))

In [56]:
p_test_i, p_test_o = get_city_trajectories(city="palo-alto", split="test", normalized=False)

In [58]:
p_test_i.shape

In [61]:
p_train_i, p_train_o = get_city_trajectories(city="palo-alto", split="train", normalized=False)

In [69]:
p_train_o[0]

In [84]:
model(torch.tensor(p_train_i[0])).shape

In [82]:
p_test_i.shape

In [86]:
predict = model(torch.tensor(p_test_i[0]))

In [91]:
predict[0][0][0]

In [92]:
palo_df = pd.DataFrame()
for i in range(len(p_test_i)):
    
    predict = model(torch.tensor(p_test_i[i]))
    print("predicts for " + str(i) + " is done")
    temp_dict = {}
    for j in range(0, 120, 2):
        col_x = 'v' + str(j)
        col_y = 'v' + str(j+1)
        row = j // 2
        temp_dict[col_x] = predict[0][row][0].item()
        temp_dict[col_y] = predict[0][row][1].item()
    
    temp_idx_name = str(i) + "_" + "palo-alto"
    temp_final = pd.DataFrame(temp_dict, index=[temp_idx_name])
    palo_df = pd.concat([palo_df, temp_final])

In [94]:
palo_df.reset_index(level=0).rename(columns={"index": "ID"}).to_csv("model5_palo_df.csv", index=False)